In [ ]:
library(dplyr)

以下 Lab 改編自 Wush Wu 的機器學習教材

這個 Lab 要介紹從統計邁入機器學習的敲門磚

利用 R 建立線性模型、學習參數、進行預測以及選模

機器學習所建立的模型通常有一個要預測的標的，稱之為標籤 (label)，統計學上稱之為反應變數 (response variable)

標籤可以是數值型變數或是類別型變數

預測數值型變數的模型稱之為回歸模型 (regression model)

預測類別型變數的則稱之為分類模型，又常簡稱為分類器 (classification model, classifier)

而在模型中，用來預測標籤的資料屬性，一般稱之為特徵 (feature)，統計學上稱之為解釋變數 (explanatory variable)

在 R 裡面，一般而言會將資料整理成包含特徵與標籤的 data.frame 的型態

例如 cars 資料集

In [ ]:
head(cars)

cars 資料集中就是登載了車速與煞停距離的資料表

而其中車速為影響煞停距離的原因之一

所以在此資料集中，車速為特徵、煞停距離為標籤

In [ ]:
# 這邊我們可以畫一張散佈圖 (scatter plot) 來觀察特徵與標籤之間的關係
plot(dist ~ speed, cars)

In [ ]:
# 由於 speed 與 dist 之間的關係看起來應該是線性的
# 所以我們可以嘗試建立線性模型來闡明 車速 與 煞停距離 之間的關係
# 請各位參照 lm 的 help

model <- lm(dist ~ speed, cars)

In [ ]:
# 建模以及繪圖時使用 dist ~ speed 是 R 的 formula 物件
# formula物件的目的是描述數據間的關係
# ~ 左方的是反應變數，也是我們感興趣的變數名
# ~ 右方的是解釋變數，也就是會影響反應變數的因素
# 請各位以 class、mode、str 等指令觀察 model 的資料結構

無論是回歸模型或是分類器

基本上就是在描述標籤與特徵，也就是反應變數與解釋變數間的關係

In [ ]:
# 由於這個模型中的解釋變數只有一個
# 所以我們可以直接將線條描繪在散佈圖上，觀察模型預測結果與反應變數間的關係
plot(dist~speed, cars)
abline(model, col = "red")

In [ ]:
# 接下來看看模型的內容
summary(model)

In [ ]:
# lm 基本上是分析了解釋變數與反應變數之間的關係
# 將其建立為模型，以利未來單除以解釋變數來預測反應變數的值，例如

predict(model, data.frame(speed = 20))

In [ ]:
# 請參閱 predict 的 help，並且嘗試預測 車速為 0, 5, 10 ... 100 的煞停距離

In [ ]:
# 請參閱 predict.lm 的 help 獲取進一步獲取其他 value 或 argument 的資訊

In [ ]:
# 根據 predict.lm 我們可以輸出預測的標準差或是信賴區間
predict(model, data.frame(speed = 20), interval = "predict")

In [ ]:
# 接下來我們嘗試在 iris 資料集上使用線性模型
# 我們要嘗試以 iris 資料集中的其他屬性預測 Sepal.Length

model.iris1 <- lm(Sepal.Length ~ ., iris)
summary(model.iris1)

In [ ]:
# 假如單獨使用類別型變數當作解釋變數
# 則效果將會接近於分組取平均，請嘗試下列程式碼

In [ ]:
group_by(iris, Species) %>%
summarize(mean(Sepal.Length))

In [ ]:
lm(Sepal.Length ~ Species, iris) %>%
summary()

In [ ]:
# lm 除了單純利用個別的特徵進行建模以外
# 也可以考慮特徵間的交互作用
model.iris2 <- lm(Sepal.Length ~ .^2, iris)
summary(model.iris2)

In [ ]:
# 同理也可以利用 .^3 計算到三種變數的交互作用
# 或者直接輸入 Sepal.Length ~ . + Sepal.Width:Petal.Width
# 考慮單一變數以及 Sepal.Width 與 Petal.Width 間的交互作用

In [ ]:
# 透過計算所謂的 residual 也就是預測值與實際值之間的差
# 可以評估模型的好壞
# 請各位參照 residual 指令，計算 model.iris1 以及 model.iris2 
# 對於 Sepal.Length 預測的平方和，並且評估哪個模型預測得比較準

透過residuals的平方和，我們可以計算出R Squared。

R Squared是很多統計學課程中會提到的「描述線性模型表現」的指標（我們略過嚴謹的數學定義）

數學上可以證明，R Squared的值會介於0與1之間

0代表模型完全沒有解釋到目標變數在training dataset上的變化

1代表模型 100% 解釋了目標變數在training dataset的行為（同時也代表residuals都是0）

R Squared等價於「相關係數的平方」。請同學輸入`summary(m.iris2)`，這個詳細的表格會回報模型的R Squared

請參照 https://en.wikipedia.org/wiki/Coefficient_of_determination

In [ ]:
# 請各位建立一個有所有三次交互作用項的模型
# 並且計算該模型 residuals 的平方和為多少

實務上，這邊的 R Squared並不代表模型比較「好」（例如：預測的比較準）

因為這裡的 R Squared 單純只是評估模型在 training dataset 上的表現

「不代表」模型在training dataset以外的資料上會表現的比較好

假如模型在 training dataset 上表現很好，但是在 training data set 以外的資料表現得很差

這種狀況稱為過適配 overfitting，一般而言越複雜的模型越有過適配的風險

而透過方法挑選出在 training dataset 以外的資料上比較好的模型，稱為「選模」

In [ ]:
# 統計學提供了一些統計指標，在只使用training dataset的狀況下
# 去預測模型在在 training dataset 以外的資料上的表現
# R 則有提供相關的實作，甚至連選模的過程都實作了，請嘗試下列程式碼
m.iris <- step(model.iris2, steps = 1)

step 函式會去計算 AIC (The Akaike information criterion) 來評估模型在 testing dataset 的好壞

簡單來說，就是評估過適配的風險，AIC越小，模型就越好

選模過程的每一個步驟都會嘗試增加或刪減模型中的一個特徵，並且觀察模型的 AIC 會不會下降

從上面的過程中我們可以看到，當拿掉：`Sepal.Width:Species`時，AIC會變低 (原本的AIC，即是<none>所對應的那一列) 

而且是這一系列嘗試中，AIC最低的。所以R 就會將`m.iris2`的解釋變數組合

從「所有的二次交互作用」(`.^2`)變成「所有的二次交互作用扣除Sepal.Width:Species」(`.^2 - Sepal.Width:Species`)

In [ ]:
step(model.iris1)

In [ ]:
# 假如不特別指定 steps 函式預設的嘗試上限是 1,000 次
# 請各位試試看不限制 step 數的結果

In [ ]:
# 請完成下列練習
#install.packages("mlbench")
library(mlbench)
data(BostonHousing)

# 同學應該先了解這個資料集。請輸入：

# ?BostonHousing # 了解資料的背景
# sapply(BostonHousing, class) # 看看每欄的型態
# summary(BostonHousing) # 看看數字分佈
# nrow(BostonHousing) # 看看資料個數

In [ ]:
# 這邊我們要建立一個模型預測波士頓的房屋價格，也就是其中的 medv 欄位
# 請嘗試建立一個 R Squared 超過0.9的模型

In [ ]:
# 請嘗試建立一個 AIC 低於 1089 的模型

進一步我們要繼續介紹 R 在廣義線性模型上的實作。

傳統的線性迴歸分析中，目標變數通常是沒有範圍限制的數值型變數

當目標變數是非負整數（例：來客數）、或是類別型變數（例：有無點擊）時

我們可以運用廣義線性模型來做學習。

In [ ]:
# 請各位參照 glm 的 help

In [ ]:
# 原始的線性模型假設模型的反應變數其分布為常態分佈
# 而廣義線性模型則容許不同的分佈
# 而在 R 中，上述設定是透過 family 這項參數達成的
# 請嘗試下列程式碼

glm(dist ~ speed, data = cars, family = "gaussian")

In [ ]:
# 使用下列程式碼載入資料集
data(Ionosphere)

In [ ]:
# 除了使用統計理論如 AIC 來評估模型是否適用於測試集以外的資料
# 通常的作法是會將既有的資料分割為 training、testing (or validation) dataset
# 這邊我們嘗試將資料集中八成的欄位用於訓練，剩下兩成用於測試
i.trn = sample(1:nrow(Ionosphere), 0.8*nrow(Ionosphere))
df.trn = Ionosphere[i.trn,]
df.test = Ionosphere[-i.trn,]

# 同學們可以觀察一下兩張資料表

In [ ]:
# 資料表中的 V2 的數值沒有變化，所以我們就不用來建模
# reformulate 可以產生 formula 的字串
# 我們將 Class 當作反應變數，V2 以外的其他特徵當作解釋變數產生 formula
names.ftr = setdiff(names(Ionosphere), c("V2", "Class"))
f <- reformulate(names.ftr, "Class")

In [ ]:
# 觀察 Class 會發現這是一個二元變數
# 所以我們以反應變數為 binomial 分佈建立模型
# 在 glm 中以 family binomial 建模，就表示要建立 logistic regression model
m1 <- glm(formula = f, family = "binomial", data = df.trn)

# Warning message "glm.fit: fitted probabilities numerically 0 or 1 occurred"
# 是因為logistic regression在某些類別組合中目標變數全部是相同類別時，會有數值問題，在此先略過

In [ ]:
# 利用 predict function 產出預測結果
p1.train <- predict(m1, df.trn, type = "response")
plot(Class ~ p1.train, data = df.trn)

In [ ]:
# 接著觀察在 training dataset 的表現
p1 <- predict(m1, df.test, type = "response")
plot(Class ~ p1, data = df.test)

In [ ]:
# 在此我們同樣可以讓特徵交互作用進而嘗試提升模型預測的能力
# 下列 function 定義出自 http://stackoverflow.com/a/29691154/1182304>
# 可以幫助我們產生二次互動的 formula 字串
interact_rhs<-function(x) {
    x[[length(as.list(x))]] <- bquote((.(x[[length(as.list(x))]]))^2)
    x
}
f2 <- interact_rhs(f)

In [ ]:
m2 <- glm(formula = f2, family = "binomial", data = df.trn)
p2.train <- predict(m2, df.trn, type = "response")
plot(Class ~ p2.train, data = df.trn)

這邊我們會發現出現了完美的結果，但這其實是因為參數量過大造成的

In [ ]:
p2 <- predict(m2, df.test, type = "response")
plot(Class ~ p2, data = df.test)

從結果來看，基本上找不出一個門檻可以良好地將好或壞分開

也就是說這個模型雖然在訓練集上表現良好，但是在測試集上表現卻很差

也就是說發生了 overfitting

我們也可以用一些數值指標來比較p1與p2的表現

一種常用的指標是Logarithmic Loss：- (y * log(p) + (1 - y) * log(1 - p)) 

這裡的y是以0或1來代表分類結果。p則是預測發生1的機率

當完美預測（y = 1 時 p = 1, y = 0時p = 0）Logarithmic Loss的結果會趨近於0

當結果越不準確，Logarithmic Loss會越大

In [ ]:
y <- df.test$Class == "good"

-sum(y * log(p1) + (1 - y) * log(1-p1))
-sum(y * log(p2) + (1 - y) * log(1-p2))

In [ ]:
# 請完成下列練習
# 方便起見，同學可以使用這個函數計算 Logarithmic Loss
logloss <- function(y, p, tol = 1e-4) {
  # tol 的用途是避免對0取log所導致的數值問題
  p[p < tol] <- tol
  p[p > 1 - tol] <- 1 - tol
  -sum(y * log(p) + (1 - y) * log(1-p))
}

# 請找出一個在df.test上的logloss小於24.5的模型
